**Load the Data**

In [ ]:
import csv
import random
import numpy as np
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#panggil dataset yang telah disimpan di Google Drive pribadi
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#explore the dataset
with open("/content/drive/MyDrive/Pribadi/data_feedbacks.csv", 'r') as csvfile:
    print(f"First line (header) looks like this:\n\n{csvfile.readline()}")
    print(f"Each data point looks like this:\n\n{csvfile.readline()}")

**Removing Stopwords**

In [838]:
pip install sastrawi

In [839]:
# GRADED FUNCTION: remove_stopwords
def remove_stopwords(ulasan):
    # List of stopwords
    stopwords = ["yang", "untuk", "pada", "ke", "para", "namun", "menurut", "antara", "dia", "dua", "ia", "seperti", 
              "jika", "sehingga", "kembali", "dan", "karena", "kepada", "oleh", "saat", "harus", "sementara", 
              "setelah", "belum", "kami", "sekitar", "bagi", "serta", "di", "dari", "telah", "sebagai", "masih", 
              "hal", "ketika", "adalah", "itu", "dalam", "bisa", "bahwa", "atau", "hanya", "kita", "dengan", "akan", 
              "juga", "ada", "mereka", "sudah", "saya", "terhadap", "secara", "agar", "lain", "anda", "begitu", "mengapa", 
              "kenapa", "yaitu", "yakni", "daripada", "itulah", "lagi", "maka", "tentang", "demi", "dimana", "kemana",
              "pula", "sambil", "sebelum", "sesudah", "supaya", "guna", "kah", "pun", "sampai", "sedangkan", "selagi",
              "sementara", "tetapi", "apakah", "kecuali", "sebab", "selain", "seolah", "seterusnya", "tanpa", "agak",
              "boleh", "dapat", "dsb", "dst", "dll", "dahulu", "dulunya", "anu", "demikian", "tapi", "ingin", "juga",
              "nggak", "mari", "nanti", "melainkan", "oh", "ok"]
    
    # Sentence converted to lowercase-only
    ulasan = ulasan.lower()
    
    ### START CODE HERE
    
    # Use this to read file content as a stream:
    words = ulasan.split()
    ulasan = []
    for r in words:
        if not r in ulasan:
            ulasan.append(r)
            
    ulasan=" ".join(ulasan)
    ### END CODE HERE
    return ulasan

**convert to lowercase, strip and remove punctuations**

In [840]:
def preprocess(ulasan):
    ulasan = ulasan.lower() 
    ulasan = ulasan.strip()  
    ulasan = re.compile('<.*?>').sub('', ulasan) 
    ulasan = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', ulasan)  
    ulasan = re.sub('\s+', ' ', ulasan)  
    ulasan = re.sub(r'\[[0-9]*\]',' ',ulasan) 
    ulasan = re.sub(r'[^\w\s]', '', str(ulasan).lower().strip())
    ulasan = re.sub(r'\d',' ',ulasan) 
    ulasan = re.sub(r'\s+',' ',ulasan) 
    return ulasan

**Reading the raw data**

In [841]:
def parse_data_from_file(filename):
    ulasan = []
    label = []
    with open(filename, 'r') as csvfile:
        ### START CODE HERE
        skip = True
        if skip:
            skip = False
            reader = csv.reader(csvfile, delimiter=',')
            next(reader)

            for row in reader:
                row[0] = remove_stopwords(row[0])
                row[1] = remove_stopwords(row[1])
                label.append(row[0])
                ulasan.append(row[1])
            
        ### END CODE HERE
    return ulasan, label

In [842]:
# Test your function
ulasan, label = parse_data_from_file("/content/drive/MyDrive/Pribadi/data_feedbacks.csv")

print(f"There are {len(ulasan)} sentences in the dataset.\n")
print(f"First sentence has {len(ulasan[0])} words (after removing stopwords).\n")
print(f"There are {len(label)} labels in the dataset.\n")
print(f"The first 5 labels are {label[:5]}")

There are 200 sentences in the dataset.

First sentence has 323 words (after removing stopwords).

There are 200 labels in the dataset.

The first 5 labels are ['1', '1', '1', '1', '0']


**Final Pre-Processing**

In [844]:
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [845]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [846]:
df['clean_text'] = df['ulasan'].apply(lambda x: finalpreprocess(x))
df.head()

,label,ulasan,clean_text
0,1.0,"Secara keseluruhan saya menyukai aplikasi ini,...",cara seluruh saya suka aplikasi ini tetapi mas...
1,1.0,Saya beri 4 bintang karena versi PC memiliki f...,saya beri bintang karena versi pc milik fitur ...
2,0.0,1. Autoscroll ke bawah tidak dapat dinonaktifk...,autoscroll ke bawah tidak dapat nonaktif yang ...
3,1.0,Benar-benar waktu yang baik menggunakan aplika...,benar benar waktu yang baik guna aplikasi ini ...
4,1.0,Ini adalah aplikasi yang bagus dan sangat muda...,ini adalah aplikasi yang bagus dan sangat muda...


**Defining some useful global variables**

In [847]:
EMBEDDING_DIM = 100
MAXLEN = 16
TRUNCATING = 'post'
PADDING = 'post'
OOV_TOKEN = "<OOV>"
MAX_EXAMPLES = 160000
TRAINING_SPLIT = 0.9

In [848]:
def train_val_split(ulasan, label, training_split):
    
    ### START CODE HERE
    
    # Compute the number of ulasan that will be used for training (should be an integer)
    train_size = int(len(ulasan)*training_split)

    # Split the sentences and labels into train/validation splits
    train_ulasan = ulasan[:train_size]
    train_label = label[:train_size]

    validation_ulasan = ulasan[train_size:]
    validation_label = label[train_size:]
    
    ### END CODE HERE
    
    return train_ulasan, validation_ulasan, train_label, validation_label

In [849]:
# Test your function
train_ulasan, validation_ulasan, train_label, validation_label = train_val_split(ulasan, label, TRAINING_SPLIT)

print(f"There are {len(train_ulasan)} sentences for training.\n")
print(f"There are {len(train_label)} labels for training.\n")
print(f"There are {len(validation_ulasan)} sentences for validation.\n")
print(f"There are {len(validation_label)} labels for validation.")

There are 180 sentences for training.

There are 180 labels for training.

There are 20 sentences for validation.

There are 20 labels for validation.


**Tokenization - Sequences, truncating and padding**

In [850]:
def fit_tokenizer(train_ulasan, oov_token):
    
    ### START CODE HERE
    
    # Instantiate the Tokenizer class, passing in the correct values for num_words and oov_token
    tokenizer = Tokenizer(num_words = len(train_ulasan), oov_token="<OOV>")
    
    # Fit the tokenizer to the training sentences
    tokenizer.fit_on_texts(train_ulasan)
    
    ### END CODE HERE
    
    return tokenizer

In [851]:
# Test your function
tokenizer = fit_tokenizer(train_ulasan, OOV_TOKEN)

word_index = tokenizer.word_index
VOCAB_SIZE = len(word_index)

print(f"Vocabulary contains {VOCAB_SIZE} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index else "<OOV> token NOT included in vocabulary")
print(f"\nindex of word 'i' should be {word_index['i']}")

Vocabulary contains 1401 words

<OOV> token included in vocabulary

index of word 'i' should be 1114


In [852]:
def seq_pad_and_trunc(ulasan, tokenizer, padding, truncating, maxlen):
    
    ### START CODE HERE
       
    # Convert sentences to sequences
    ulasan = tokenizer.texts_to_sequences(ulasan)
    
    # Pad the sequences using the correct padding, truncating and maxlen
    pad_trunc_sequences = pad_sequences(ulasan, padding='post', truncating='post', maxlen=maxlen)
    
    ### END CODE HERE
    
    return pad_trunc_sequences

In [853]:
# Test your function
train_pad_trunc_seq = seq_pad_and_trunc(train_ulasan, tokenizer, PADDING, TRUNCATING, MAXLEN)
val_pad_trunc_seq = seq_pad_and_trunc(validation_ulasan, tokenizer, PADDING, TRUNCATING, MAXLEN)

print(f"Padded and truncated training sequences have shape: {train_pad_trunc_seq.shape}\n")
print(f"Padded and truncated validation sequences have shape: {val_pad_trunc_seq.shape}")

Padded and truncated training sequences have shape: (180, 16)

Padded and truncated validation sequences have shape: (20, 16)


In [854]:
train_label = np.array(train_label)
validation_label = np.array(validation_label)

**Using pre-defined Embeddings**

In [855]:
# Define path to file containing the embeddings
GLOVE_FILE = './data/glove.6B.100d.txt'

# Initialize an empty embeddings index dictionary
GLOVE_EMBEDDINGS = {}

# Read file and fill GLOVE_EMBEDDINGS with its contents
with open(GLOVE_FILE) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        GLOVE_EMBEDDINGS[word] = coefs

FileNotFoundError: ignored